In [ ]:
!pip install torch torchvision --quiet
!pip install kaggle --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder, CIFAR10
from torchvision.models import resnet18
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
from google.colab import files
print("Upload your kaggle.json file")
uploaded = files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("Kaggle configured")

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats
!unzip -q dogs-vs-cats.zip -d cats_vs_dogs
!unzip -q cats_vs_dogs/train.zip -d cats_vs_dogs/
print("Dataset downloaded")

In [ ]:
source = Path('cats_vs_dogs/train')
target = Path('cats_vs_dogs/organized')

cats_dir = target / 'cats'
dogs_dir = target / 'dogs'
cats_dir.mkdir(parents=True, exist_ok=True)
dogs_dir.mkdir(parents=True, exist_ok=True)

count_cats = 0
count_dogs = 0

for img in source.glob('*.jpg'):
    if 'cat' in img.name:
        shutil.copy(img, cats_dir / img.name)
        count_cats += 1
    elif 'dog' in img.name:
        shutil.copy(img, dogs_dir / img.name)
        count_dogs += 1

print(f"Organized {count_cats} cats and {count_dogs} dogs")

In [ ]:
def get_loaders(dataset_name, batch_size=32):
    if dataset_name == 'cats_vs_dogs':
        transform_train = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        transform_test = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        dataset = ImageFolder('cats_vs_dogs/organized', transform=transform_train)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_data, val_data = random_split(dataset, [train_size, val_size])
        val_data.dataset.transform = transform_test
        num_classes = 2
        
    else:
        transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
        ])
        
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
        ])
        
        train_data = CIFAR10('./data', train=True, download=True, transform=transform_train)
        val_data = CIFAR10('./data', train=False, download=True, transform=transform_test)
        num_classes = 10
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, val_loader, num_classes

In [ ]:
def show_samples(dataset_name):
    train_loader, _, _ = get_loaders(dataset_name, 16)
    images, labels = next(iter(train_loader))
    
    if dataset_name == 'cats_vs_dogs':
        classes = ['Cat', 'Dog']
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    else:
        classes = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
        mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
        std = torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    
    fig, axes = plt.subplots(2, 8, figsize=(16, 4))
    for i in range(16):
        ax = axes[i // 8, i % 8]
        img = images[i] * std + mean
        img = torch.clamp(img, 0, 1).permute(1, 2, 0).numpy()
        ax.imshow(img)
        ax.set_title(classes[labels[i]])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

show_samples('cats_vs_dogs')
show_samples('cifar10')

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes, activation='relu', input_size=128):
        super(CNN, self).__init__()
        
        if activation == 'relu':
            self.act = nn.ReLU()
        elif activation == 'tanh':
            self.act = nn.Tanh()
        else:
            self.act = nn.LeakyReLU(0.2)
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        feature_size = (input_size // 16) * (input_size // 16) * 256
        
        self.fc1 = nn.Linear(feature_size, 512)
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.drop2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = self.pool(self.act(self.bn1(self.conv1(x))))
        x = self.pool(self.act(self.bn2(self.conv2(x))))
        x = self.pool(self.act(self.bn3(self.conv3(x))))
        x = self.pool(self.act(self.bn4(self.conv4(x))))
        
        x = x.view(x.size(0), -1)
        x = self.drop1(self.act(self.fc1(x)))
        x = self.drop2(self.act(self.fc2(x)))
        x = self.fc3(x)
        
        return x

In [ ]:
def init_weights(model, method='xavier'):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            if method == 'xavier':
                nn.init.xavier_uniform_(m.weight)
            elif method == 'kaiming':
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            else:
                nn.init.uniform_(m.weight, -0.1, 0.1)
            
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
    
    return model

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0.0
    best_state = None
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100. * correct / total
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f'Epoch {epoch+1}: Loss={train_loss:.4f}, Acc={train_acc:.2f}%, Val Loss={val_loss:.4f}, Val Acc={val_acc:.2f}%')
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = model.state_dict().copy()
    
    model.load_state_dict(best_state)
    return model, history, best_acc

In [ ]:
def run_experiments(dataset_name, epochs=10):
    activations = ['relu', 'tanh', 'leaky_relu']
    inits = ['xavier', 'kaiming', 'random']
    opts = ['sgd', 'adam', 'rmsprop']
    
    results = []
    best_acc = 0
    best_config = None
    best_model = None
    
    train_loader, val_loader, num_classes = get_loaders(dataset_name)
    input_size = 128 if dataset_name == 'cats_vs_dogs' else 32
    
    exp_num = 0
    total = len(activations) * len(inits) * len(opts)
    
    for act in activations:
        for init in inits:
            for opt in opts:
                exp_num += 1
                print(f"\n{'='*80}")
                print(f"Experiment {exp_num}/{total}: {dataset_name} | {act} | {init} | {opt}")
                print(f"{'='*80}")
                
                model = CNN(num_classes, act, input_size)
                model = init_weights(model, init)
                model = model.to(device)
                
                criterion = nn.CrossEntropyLoss()
                
                if opt == 'sgd':
                    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
                elif opt == 'adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.001)
                else:
                    optimizer = optim.RMSprop(model.parameters(), lr=0.001)
                
                trained_model, history, acc = train_model(model, train_loader, val_loader, criterion, optimizer, epochs)
                
                config = {
                    'activation': act,
                    'initialization': init,
                    'optimizer': opt,
                    'best_accuracy': acc,
                    'history': history
                }
                
                results.append(config)
                
                if acc > best_acc:
                    best_acc = acc
                    best_config = config
                    best_model = trained_model
    
    return results, best_config, best_model

In [ ]:
print("Running Cats vs Dogs experiments...")
cats_results, cats_best, cats_model = run_experiments('cats_vs_dogs', epochs=10)

print(f"\n{'='*80}")
print(f"CATS VS DOGS BEST: {cats_best['activation']} | {cats_best['initialization']} | {cats_best['optimizer']}")
print(f"Accuracy: {cats_best['best_accuracy']:.2f}%")
print(f"{'='*80}")

In [ ]:
print("Running CIFAR-10 experiments...")
cifar_results, cifar_best, cifar_model = run_experiments('cifar10', epochs=10)

print(f"\n{'='*80}")
print(f"CIFAR-10 BEST: {cifar_best['activation']} | {cifar_best['initialization']} | {cifar_best['optimizer']}")
print(f"Accuracy: {cifar_best['best_accuracy']:.2f}%")
print(f"{'='*80}")

In [ ]:
def plot_comparison(results, dataset_name):
    acts = list(set([r['activation'] for r in results]))
    inits = list(set([r['initialization'] for r in results]))
    opts = list(set([r['optimizer'] for r in results]))
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    act_accs = {a: np.mean([r['best_accuracy'] for r in results if r['activation'] == a]) for a in acts}
    axes[0].bar(act_accs.keys(), act_accs.values(), color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    axes[0].set_title('Activation Functions')
    axes[0].set_ylabel('Accuracy (%)')
    axes[0].set_ylim([0, 100])
    axes[0].grid(axis='y', alpha=0.3)
    
    init_accs = {i: np.mean([r['best_accuracy'] for r in results if r['initialization'] == i]) for i in inits}
    axes[1].bar(init_accs.keys(), init_accs.values(), color=['#95E1D3', '#F38181', '#AA96DA'])
    axes[1].set_title('Initializations')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_ylim([0, 100])
    axes[1].grid(axis='y', alpha=0.3)
    
    opt_accs = {o: np.mean([r['best_accuracy'] for r in results if r['optimizer'] == o]) for o in opts}
    axes[2].bar(opt_accs.keys(), opt_accs.values(), color=['#FECA57', '#48DBFB', '#FF9FF3'])
    axes[2].set_title('Optimizers')
    axes[2].set_ylabel('Accuracy (%)')
    axes[2].set_ylim([0, 100])
    axes[2].grid(axis='y', alpha=0.3)
    
    plt.suptitle(f'{dataset_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_comparison(cats_results, 'Cats vs Dogs')
plot_comparison(cifar_results, 'CIFAR-10')

In [ ]:
def plot_history(history, title):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].plot(history['train_loss'], label='Train', linewidth=2, color='#FF6B6B')
    axes[0].plot(history['val_loss'], label='Val', linewidth=2, color='#4ECDC4')
    axes[0].set_title('Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    axes[1].plot(history['train_acc'], label='Train', linewidth=2, color='#FF6B6B')
    axes[1].plot(history['val_acc'], label='Val', linewidth=2, color='#4ECDC4')
    axes[1].set_title('Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.suptitle(title, fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_history(cats_best['history'], 'Cats vs Dogs - Best Model')
plot_history(cifar_best['history'], 'CIFAR-10 - Best Model')

In [ ]:
def train_resnet(dataset_name, epochs=10):
    train_loader, val_loader, num_classes = get_loaders(dataset_name)
    
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    model, history, acc = train_model(model, train_loader, val_loader, criterion, optimizer, epochs)
    return model, history, acc

In [ ]:
print("Training ResNet-18 on Cats vs Dogs...")
resnet_cats, resnet_cats_hist, resnet_cats_acc = train_resnet('cats_vs_dogs', 10)
print(f"ResNet-18 Accuracy: {resnet_cats_acc:.2f}%")

In [ ]:
print("Training ResNet-18 on CIFAR-10...")
resnet_cifar, resnet_cifar_hist, resnet_cifar_acc = train_resnet('cifar10', 10)
print(f"ResNet-18 Accuracy: {resnet_cifar_acc:.2f}%")

In [ ]:
def compare_models(custom_acc, resnet_acc, dataset_name):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    models = ['Custom CNN', 'ResNet-18']
    accs = [custom_acc, resnet_acc]
    
    bars = ax.bar(models, accs, color=['#FF6B6B', '#4ECDC4'], alpha=0.8, width=0.5)
    
    ax.set_ylabel('Accuracy (%)')
    ax.set_title(f'{dataset_name}')
    ax.set_ylim([0, 100])
    ax.grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}%',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Custom CNN: {custom_acc:.2f}%")
    print(f"ResNet-18: {resnet_acc:.2f}%")
    print(f"Difference: {abs(resnet_acc - custom_acc):.2f}%")

compare_models(cats_best['best_accuracy'], resnet_cats_acc, 'Cats vs Dogs')
compare_models(cifar_best['best_accuracy'], resnet_cifar_acc, 'CIFAR-10')

In [ ]:
os.makedirs('saved_models', exist_ok=True)

torch.save({'model_state_dict': cats_model.state_dict(), 'config': cats_best, 'accuracy': cats_best['best_accuracy']}, 'saved_models/cats_dogs_best_cnn.pth')
torch.save({'model_state_dict': cifar_model.state_dict(), 'config': cifar_best, 'accuracy': cifar_best['best_accuracy']}, 'saved_models/cifar10_best_cnn.pth')
torch.save({'model_state_dict': resnet_cats.state_dict(), 'accuracy': resnet_cats_acc}, 'saved_models/cats_dogs_resnet18.pth')
torch.save({'model_state_dict': resnet_cifar.state_dict(), 'accuracy': resnet_cifar_acc}, 'saved_models/cifar10_resnet18.pth')

print("Models saved")

In [ ]:
results_summary = {
    'cats_vs_dogs': {
        'best_config': {'activation': cats_best['activation'], 'initialization': cats_best['initialization'], 'optimizer': cats_best['optimizer'], 'accuracy': float(cats_best['best_accuracy'])},
        'resnet18_accuracy': float(resnet_cats_acc),
        'all_experiments': [{'activation': r['activation'], 'initialization': r['initialization'], 'optimizer': r['optimizer'], 'accuracy': float(r['best_accuracy'])} for r in cats_results]
    },
    'cifar10': {
        'best_config': {'activation': cifar_best['activation'], 'initialization': cifar_best['initialization'], 'optimizer': cifar_best['optimizer'], 'accuracy': float(cifar_best['best_accuracy'])},
        'resnet18_accuracy': float(resnet_cifar_acc),
        'all_experiments': [{'activation': r['activation'], 'initialization': r['initialization'], 'optimizer': r['optimizer'], 'accuracy': float(r['best_accuracy'])} for r in cifar_results]
    }
}

with open('saved_models/experiment_results.json', 'w') as f:
    json.dump(results_summary, f, indent=4)

print("Results saved to JSON")

In [ ]:
print("="*100)
print(" " * 40 + "FINAL REPORT")
print("="*100)

print("\nCATS VS DOGS:")
print(f"  Best: {cats_best['activation']} | {cats_best['initialization']} | {cats_best['optimizer']}")
print(f"  Custom CNN: {cats_best['best_accuracy']:.2f}%")
print(f"  ResNet-18: {resnet_cats_acc:.2f}%")

print("\nCIFAR-10:")
print(f"  Best: {cifar_best['activation']} | {cifar_best['initialization']} | {cifar_best['optimizer']}")
print(f"  Custom CNN: {cifar_best['best_accuracy']:.2f}%")
print(f"  ResNet-18: {resnet_cifar_acc:.2f}%")

print("\n" + "="*100)

In [ ]:
!zip -r saved_models.zip saved_models/
from google.colab import files
files.download('saved_models.zip')
print("Downloaded")